![title](imgs/explicationGlobalesWP.png)
L'objectif de ce notebook est de présenter un exemple de fonctionnement de **pipelines de classification d'entités** au sein d'un moteur NLP.

In [1]:
import csv
from rasa_nlu.extractors.crf_entity_extractor import CRFEntityExtractor
import sklearn_crfsuite
import nltk
from nltk.tag.hmm import HiddenMarkovModelTrainer
from sklearn.model_selection import train_test_split
from rasa_nlu.tokenizers import Tokenizer, Token
from rasa_nlu.training_data import Message
import random, spacy
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leclerc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Une telle pipeline se divise en trois étapes :
- Tokenization (séparation de la phrase en mot)
- Pré-traitement (optionel) : association d'un label syntaxique à chaque mot
- Featurization (pour le classificateur CRF seulement)
- Classification : permet d'associer l'entité la plus probable à chaque mot

------

![title](imgs/explicationMLEntitiesWP.png)

----

L'objet **CRFEntityExtractor** nous aidera à effectuer l'étape de featurization pour l'extracteur **CRF**

In [2]:
crfent_extractor = CRFEntityExtractor()
nlp = spacy.load('en', disable = ['tagger','parser','ner','textcat'])

# Chargement des données
Les données sont issues d'un concours de data science sur Kaggle (https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus), il est possible d'explorer plus en détail le dataset en utilisant l'outil http://gmb.let.rug.nl/explorer/ ). On les charge en lisant le csv qui contient les phrases tokenizées avec **l'entité associé à chaque mot.** 

Les entités présentes dans ce corpus sont les suivantes :

    geo = Geographical Entity
    org = Organization
    per = Person
    gpe = Geopolitical Entity
    tim = Time indicator
    art = Artifact
    eve = Event
    nat = Natural Phenomenon
   
Elles sont précédées par B (**Beginning**) ou I (**Inside**), ce qui signifie que le mot commence une entité de plusieur mots / est une entité d'un mot ou qu'il est à l'intérieur d'une entité / termine une entité de plusieur mots


In [4]:
csvfileName = 'ner_dataset.csv'

datasetCrf = []
datasetHmm = []

with open(csvfileName, 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    currentSentanceCrf = []
    currentSentanceHmm = []
    for row in spamreader:
        text = row[1]
        text = text.decode('unicode-escape')
        entity = row[-1]
        if row[0] == '':
            currentSentanceCrf.append((text,None,entity,None))
            currentSentanceHmm.append((text,entity))
        else : 
            datasetCrf.append(currentSentanceCrf)
            datasetHmm.append(currentSentanceHmm)
            currentSentanceCrf=[]
            currentSentanceHmm=[]
            currentSentanceCrf.append((text,None,entity,None))
            currentSentanceHmm.append((text,entity))
del datasetCrf[0:2]
del datasetHmm[0:2]

In [5]:
datasetHmm[0][0:3]

[(u'Thousands', 'O'), (u'of', 'O'), (u'demonstrators', 'O')]

-----

# Extraction d'entités
## Chaîne de Markov Cachée
Le premier modèle d'extracteur d'entités est basé sur le principe de chaîne de Markov cachée (**HMM** pour  *Hidden Markov Chain*) :

L'idée est de considérer les labels des mots d'une phrase comme une suite **d'états cachés** d'une HMM, les **états observables** étant les mots en eux mêmes. On calcule alors sur un corpus la **matrice de transitions**(probabilité de passage d'un nom propre à un verbe), **d'émission** (probabilité d'avoir le mot "danse" en tant que verbe ou nom commun) ainsi que les **probabilités des états initiaux** (probabilité qu'une phrase commence par un verbe ou un nom).

Une fois qu'on a entraîné le modèle sur un ensemble de texte important, on essaye de trouver **la séquence de labels la plus probable** connaissant les paramètres présentés précédemment.


In [6]:
HMM = HiddenMarkovModelTrainer()

On divise nos données en données de test et d'entraînement :

In [7]:
xTrainHmm, xTestHmm, _,_ = train_test_split(
    datasetHmm, range(len(datasetHmm)), test_size=0.3, random_state=42)

In [8]:
xTrainHmm[23]

[(u'FIFA', 'B-org'),
 (u'is', 'O'),
 (u'unhappy', 'O'),
 (u'that', 'O'),
 (u'Nigerian', 'B-gpe'),
 (u'Sports', 'O'),
 (u'Minister', 'O'),
 (u'Musa', 'B-per'),
 (u'Mohammed', 'I-per'),
 (u'appointed', 'O'),
 (u'Segun', 'B-per'),
 (u'Odegbami', 'I-per'),
 (u'as', 'O'),
 (u'interim', 'O'),
 (u'secretary-general', 'O'),
 (u'of', 'O'),
 (u'the', 'O'),
 (u'NFA', 'B-org'),
 (u'and', 'O'),
 (u'Salisu', 'B-org'),
 (u'Abubakar', 'I-org'),
 (u'as', 'O'),
 (u'secretary', 'O'),
 (u'of', 'O'),
 (u'the', 'O'),
 (u'Nigerian', 'B-gpe'),
 (u'Premier', 'O'),
 (u'League', 'O'),
 (u'board', 'O'),
 (u'on', 'O'),
 (u'December', 'B-tim'),
 (u'21', 'I-tim'),
 (u'.', 'O')]

On **entraîne** le modèle avec les données divisées précedemment

In [9]:
HMMtagger = HMM.train_supervised(xTrainHmm)

On peut récupérer une **phrase aléatoire** à partir du modèle 

In [26]:
HMMtagger.random_sample(random,10)

[(u'large', 'O'),
 (u'is', 'O'),
 (u'eventually', 'O'),
 (u'skilled', 'O'),
 (u'off', 'O'),
 (u'six', 'O'),
 (u'a', 'O'),
 (u'President', 'B-per'),
 (u'"', 'O'),
 (u'military', 'O')]

## Resultats
**Testons notre modèle** sur le jeu de données de test précédent

In [27]:
HMMtagger.evaluate(xTestHmm)

0.9423280541044123

---
L'évaluation du modèle donne un F1-score de **0.94** que nous comparerons avec celui donné par l'extracteur CRF

On peut récupérer les données liées au modèle et essayer de faire quelques tests, avec :
- **P** : matrice de probabilité **a priori** (probabilité d'avoir l'état i comme état caché du premier mot d'une phrase
- **O** : matrice de proababilité d'**émission** d'un mot dans un état (danse en tant que verbe ou nom par exemple)
- **X** : matrice de probabilité de **transition** (passage d'un état i à un état j)

Ces matrices sont données avec des **log-probabilités** à la place des probabilités normales

In [28]:
symbols = HMMtagger._symbols

In [29]:
states = HMMtagger._states

In [30]:
P,O,X,S = HMMtagger._cache

In [31]:
import numpy as np

In [32]:
def mostProbableWordGivenState(stateIndex,emissionMatrix,symbols):
    emissionProbFromState = [O[stateIndex,i] for i in range(len(symbols))]
    nextWordIndex = emissionProbFromState.index(max(emissionProbFromState))
    return symbols[nextWordIndex]

In [33]:
def mostProbableNextState(lastState,transitionMatrix,states, avoidOInNextTransition = False):
    probFromLastState = [ transitionMatrix[lastState,i] for i in range(len(states))]
    if lastState == 0 or avoidOInNextTransition :
        #On évite de boucler sur des 'O' (mots associés à aucune entités) qui sont les plus probables
        return probFromLastState.index(max(probFromLastState[1:]))
    else :
        return probFromLastState.index(max(probFromLastState))

----
On affiche ici les **mots les plus probables** connaissant une entité

In [34]:
zip([states[i] for i in range(len(states))], [mostProbableWordGivenState(j,O,symbols) for j in range(len(states)) ])

[('O', u'the'),
 ('B-gpe', u'Iraqi'),
 ('B-org', u'U.S.'),
 ('I-org', u'Nations'),
 ('B-tim', u'Tuesday'),
 ('B-per', u'Mr.'),
 ('B-geo', u'U.S.'),
 ('I-tim', u'of'),
 ('I-per', u'Bush'),
 ('I-geo', u'States'),
 ('I-gpe', u'Serb'),
 ('B-eve', u'II'),
 ('I-eve', u'War'),
 ('B-nat', u'Katrina'),
 ('B-art', u'English'),
 ('I-art', u'David'),
 ('I-nat', u'Katrina')]

----
Ici, ce sont les **transitions les plus probables** qui sont affichées

In [35]:
[states[j] + ' -> ' + states[mostProbableNextState(j,X,states, avoidOInNextTransition= True)] for j in range(len(states))]

['O -> B-geo',
 'B-gpe -> B-per',
 'B-org -> I-org',
 'I-org -> I-org',
 'B-tim -> I-tim',
 'B-per -> I-per',
 'B-geo -> I-geo',
 'I-tim -> I-tim',
 'I-per -> I-per',
 'I-geo -> I-geo',
 'I-gpe -> B-per',
 'B-eve -> I-eve',
 'I-eve -> I-eve',
 'B-nat -> I-nat',
 'B-art -> I-art',
 'I-art -> I-art',
 'I-nat -> I-nat']

---
## Champs aléatoire conditionnel
Une autre méthode consiste à utiliser un classificateur basé sur la théorie des **champs aléatoires conditionnels** (Conditional Random Field, **CRF** en anglais), qui n'est autre qu'une **généralisation des chaînes de Markov cachées**. Celui-ci consiste à définir des fonctions caractéristiques / **features fonctions** qui prennent en entrée la phrase, la position du mot considéré, ainsi que son label et le label précédent.

Ces fonctions caractéristiques permettent de **calculer le score d'une séquence** donnée connaissant la phrase, la séquence donnant le score le plus important sera celle qu'on considérera pour extraire les entités.


In [ ]:
xCrf = [crfent_extractor._sentence_to_features(sent) for sent in datasetCrf]
yCrf = [crfent_extractor._sentence_to_labels(sent) for sent in datasetCrf]

In [ ]:
xCrfTrain, xCrfTest, yCrfTrain, yCrfTest = train_test_split(
    xCrf, yCrf, test_size=0.3, random_state=42)

---
Un exemple de **features** associé au mot 1 de la phrase de test 11 est donnée plus bas, on remarque les features extraites pour un mot concerne dans notre cas soit :

- Le mot précédent le mot à classifier (-1: nom de la Feature)
- Le mot en lui-même (0: nom de la Feature)
- Le mot suivant (1: nom de la Feature)

L'ensemble des features à prendre en compte ainsi que la taille de la fenêtre peuvent être paramétrer à la création du **crfent_extractor**.



In [268]:
datasetCrf[11][0:10]

[(u'The', None, 'O', None),
 (u'European', None, 'B-org', None),
 (u'Union', None, 'I-org', None),
 (u'"', None, 'O', None),
 (u'with', None, 'O', None),
 (u'U.S.', None, 'B-gpe', None),
 (u'backing', None, 'O', None),
 (u'"', None, 'O', None),
 (u'has', None, 'O', None),
 (u'threatened', None, 'O', None)]

In [271]:
datasetCrf[11][1]

(u'European', None, 'B-org', None)

In [270]:
xFull[11][1]

{'-1:low': u'the',
 '-1:title': True,
 '-1:upper': False,
 '0:bias': u'bias',
 '0:digit': False,
 '0:low': u'european',
 '0:pattern': u'N/A',
 '0:prefix2': u'Eu',
 '0:prefix5': u'Europ',
 '0:suffix2': u'an',
 '0:suffix3': u'ean',
 '0:suffix5': u'opean',
 '0:title': True,
 '0:upper': False,
 '1:low': u'union',
 '1:title': True,
 '1:upper': False}

In [200]:
CRFTagger = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        # include transitions that are possible, but not observed
        all_possible_transitions=True
)

---
On peut **entraîner** notre modèle avec les données d'entraînement précédentes :

In [203]:
CRFTagger.fit(xCrfTrain, yCrfTrain)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [206]:
CRFTagger.score(xCrfTest,yCrfTest)

0.970461331052486

---
L'extracteur CRF performe mieux que celui basé sur un HMM sur ce dataset avec un score de **0.97** contre **0.94** pour celui basé sur une HMM.

On va utiliser la bibliothèque **eli5 pour analyser le fonctionnement** du classificateur CRF (**transition** d'une entité à une autre et **importance des features** dans la classification)

In [273]:
import eli5

In [276]:
eli5.show_weights(CRFTagger,top=50)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,3.651,1.053,-3.523,1.151,-3.216,1.702,-6.575,0.799,-2.709,0.788,-2.222,1.911,-6.158,1.657,-4.745,1.957,-5.935
B-art,-0.576,-0.085,6.055,0.0,-0.1,-0.993,-1.34,-1.698,0.0,0.0,0.0,-0.046,-1.754,-1.776,-1.295,-0.114,-0.798
I-art,-0.612,-0.602,5.835,0.0,0.0,-0.263,-0.963,-0.766,0.0,0.0,0.0,-1.001,-1.164,-1.065,-1.101,-0.41,-1.077
B-eve,-1.341,0.0,-0.097,-0.529,5.488,-1.192,-1.191,-1.609,0.0,-0.216,0.0,-1.95,-1.662,-2.238,-1.305,-0.347,-1.103
I-eve,0.104,0.0,0.0,-1.95,5.998,-0.59,-0.607,-0.217,0.0,0.0,0.0,-0.554,-0.792,-1.198,-0.287,-1.739,-0.5
B-geo,0.091,-0.238,-2.391,-0.331,-2.137,-4.353,4.918,-1.086,-2.664,-0.342,-0.694,-0.877,-4.778,-1.918,-4.23,0.773,-3.31
I-geo,-0.136,0.533,-1.38,-0.466,-1.247,-2.952,4.058,-1.871,-0.928,-0.002,-0.103,-0.921,-3.541,-1.127,-2.541,0.271,-2.2
B-gpe,0.433,-0.615,-1.912,-0.133,-1.985,0.517,-3.475,-4.674,4.281,-0.073,-0.664,1.715,-4.681,-0.883,-3.591,-0.441,-2.215
I-gpe,0.022,0.0,0.0,0.0,0.0,-0.912,-0.845,-0.071,5.292,0.0,0.0,-0.525,-0.739,-1.121,-0.454,-0.752,-0.059
B-nat,-1.312,0.0,0.0,0.0,0.0,0.237,-0.527,-1.05,0.0,-0.496,6.013,-0.584,-0.603,-1.333,-0.938,-0.879,-0.151


---
On peut s'intéresser aux **features les plus influentes** dans la classification d'un mot en tant que "I-eve" : 
- le poids le plus élevé est associé au fait que le mot est suivi de caused, ce qui semble en accord avec une entité évènement
- le poid le plus important associé au mot lui-même est 0:low = 'games', ce qui s'explique par le fait que le corpus est basé en partie sur des textes parlant des jeux olympiques

---
On peut maintenant **tester nos deux extracteurs** sur la phrase 'United States of America is a powerful country as Mr. Bush said'

In [223]:
def setToken(message):
    doc = nlp(message.text)
    message.set("tokens", [Token(t.text,t.idx) for t in doc])

In [241]:
textMess = u'United States of America is a powerful country as Mr. Bush said'

In [242]:
tokenized = word_tokenize(textMess)

In [243]:
mess = Message(text=textMess)
setToken(mess)

text_data = crfent_extractor._from_text_to_crf(mess)

features = crfent_extractor._sentence_to_features(text_data)

In [244]:
CRFTagger.predict_single(features)

['B-geo',
 'I-geo',
 'I-geo',
 'I-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-per',
 'I-per',
 'O']

In [245]:
HMMtagger.best_path(tokenized)

['B-geo',
 'I-geo',
 'I-geo',
 'I-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-per',
 'I-per',
 'O']